# Relatório de Governança de Dados – Amazon Reviews (Electronics)

Notebook para perfilamento de colunas e geração de artefatos de governança.

**Correção incluída:** cálculo de `distinct` robusto para colunas com valores do tipo `numpy.ndarray`/listas (ex.: `HELPFUL`).

In [ ]:
import re
from datetime import datetime
from pathlib import Path
from typing import Any

import numpy as np
import pandas as pd


## Caminhos (ajuste se necessário)

In [ ]:
PARQUET_PATH = r"C:\\Users\\Rodrigo\\Desktop\\py\\Prjt\\DDF_TECH_122025\\notebooks\\data\\electronics_reviews_prepared.parquet"
OUT_DIR = Path(r"C:\\Users\\Rodrigo\\Desktop\\py\\Prjt\\DDF_TECH_122025\\governanca")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SAMPLE_ROWS = 200_000  # reduza se quiser acelerar


## Leitura do Parquet

In [ ]:
df = pd.read_parquet(PARQUET_PATH)
print('df:', df.shape)


## Amostragem (para acelerar métricas custosas)

In [ ]:
df_sample = df.sample(SAMPLE_ROWS, random_state=42) if SAMPLE_ROWS and len(df) > SAMPLE_ROWS else df
print('df_sample:', df_sample.shape)


## Funções auxiliares (inclui `safe_nunique`)

In [ ]:
def safe_str(x: Any, limit: int = 300) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ''
    s = str(x).replace('\n', ' ').replace('\r', ' ').strip()
    return s[:limit] + ('…' if len(s) > limit else '')

def safe_nunique(series: pd.Series) -> int:
    """Conta distintos mesmo quando a coluna contém arrays/listas (não-hashable)."""
    s = series.dropna()
    if s.empty:
        return 0
    ex = None
    for v in s.head(50).tolist():
        if v is not None:
            ex = v
            break
    if isinstance(ex, (list, tuple, np.ndarray)):
        return s.apply(lambda x: tuple(x) if isinstance(x, (list, np.ndarray)) else x).nunique(dropna=True)
    return s.nunique(dropna=True)


## (Opcional) Identificar colunas com arrays/listas

In [ ]:
array_cols = []
for col in df_sample.columns:
    s = df_sample[col].dropna()
    if not s.empty and isinstance(s.iloc[0], (list, tuple, np.ndarray)):
        array_cols.append(col)
array_cols


## Perfilamento (nulos + distintos + tipos)

In [ ]:
profiles = []
for col in df.columns:
    s = df[col]
    null_pct = s.isna().mean() * 100
    distinct = safe_nunique(df_sample[col])
    profiles.append({
        'column': col,
        'dtype': str(s.dtype),
        'null_pct': round(null_pct, 2),
        'distinct': int(distinct)
    })
df_profile = pd.DataFrame(profiles).sort_values('column')
df_profile.head(20)


## Exportar dicionário de dados (CSV)

In [ ]:
csv_path = OUT_DIR / 'dicionario_dados.csv'
df_profile.to_csv(csv_path, index=False, encoding='utf-8')
csv_path


## Gerar relatório de governança (Markdown)

In [ ]:
md_path = OUT_DIR / 'governanca_relatorio.md'
lines = []
lines.append('# Relatório de Governança de Dados\n')
lines.append(f"\nGerado em {datetime.utcnow().strftime('%Y-%m-%d %H:%M UTC')}\n")
lines.append(f"\nFonte: {PARQUET_PATH}\n")
lines.append(f"\nLinhas: {len(df):,} | Colunas: {df.shape[1]}\n")
lines.append('\n## Sumário\n')
lines.append(f"- Colunas com nulos: {(df_profile['null_pct'] > 0).sum()}\n")
lines.append(f"- Colunas com arrays/listas: {len(array_cols)} ({', '.join(array_cols) if array_cols else 'nenhuma'})\n")
lines.append('\n## Dicionário de Dados (perfilamento básico)\n')
lines.append(df_profile.to_markdown(index=False))
md_path.write_text('\n'.join(lines), encoding='utf-8')
md_path


## Próximo passo

Use `governanca_relatorio.md` como evidência para Item de Governança/Data Quality e cite no README.